In [11]:
import pandas as pd
import numpy as np
import datetime
from datetime import date
import calendar

df = pd.read_csv('Arrest_Data_from_2010_to_Present.csv') # read in the csv file

In [2]:
#delete columns not relevant to analysis
df.drop(['Report ID','Area ID','Charge Group Code','Location'],axis=1,inplace=True)

In [3]:
#need to cleanup the time field...it is stored like 645 instead of 06:45
df_cleansed = df

#convert float to string
df_cleansed['Time'] = df_cleansed['Time'].astype(str) 

#get rid of decimals
df_cleansed['Time'] = df_cleansed['Time'].str.split(".", expand=True)[0] 

#convert missing to 0000
df_cleansed['Time'] = df_cleansed['Time'].replace(to_replace="nan",value="0000") 

#treat 0 as missing and convert to 0000
df_cleansed['Time'] = df_cleansed['Time'].replace(to_replace="0",value="0000") 

#2400 is not a valid time, converting to 0001 so it isn't the same as missing
df_cleansed['Time'] = df_cleansed['Time'].replace(to_replace="2400",value="0001") 

#split the time string to get the appropriate digits that correspond to hours and minutes
df_cleansed['Hour'] = np.where(df_cleansed['Time'].str.len() == 4,df_cleansed['Time'].str[-4:2],np.where(df_cleansed['Time'].str.len() == 3,df_cleansed['Time'].str[-3:1],"00"))
df_cleansed['Minute'] = df_cleansed['Time'].str[-2:4]

#put hour and minute back together in time format
df_cleansed['NewTime'] = pd.to_datetime(df_cleansed['Hour'] + ':' + df_cleansed['Minute'] + ':00',format='%H:%M:%S').dt.time

In [4]:
#need to clean up cross street field

#remove duplicate whitespaces
df_cleansed['Cross Street'] = df_cleansed['Cross Street'].replace('\s+',' ',regex=True)
df_cleansed['Address'] = df_cleansed['Address'].replace('\s+',' ',regex=True)

#if all digits are numeric, nullify
df_cleansed['Address New'] = np.where(df_cleansed["Address"].str.isdigit() == True,np.nan, df_cleansed["Address"])
df_cleansed['Cross Street New'] = np.where(df_cleansed["Cross Street"].str.isdigit() == True,np.nan, df_cleansed["Cross Street"])

df_cleansed['Address_first_word'] = df_cleansed['Address'].str.split(n=1).str[0]
df_cleansed['Street'] = np.where(df_cleansed['Address_first_word'].str.isdigit() == True,df_cleansed['Address'].str.split(n=1).str[1],df_cleansed['Address'])

df_cleansed['Cross_street_first_word'] = df_cleansed['Cross Street'].str.split(n=1).str[0]
df_cleansed['CrossStreet'] = np.where(df_cleansed['Cross_street_first_word'].str.isdigit() == True,df_cleansed['Cross Street'].str.split(n=1).str[1],df_cleansed['Cross Street'])

In [5]:
#delete columns not relevant to analysis
df_cleansed.drop(['Time','Hour','Minute','Address','Cross Street','Address New','Cross Street New','Address_first_word','Cross_street_first_word'],axis=1,inplace=True)

#add year column
df_cleansed['Date'] = pd.to_datetime(df_cleansed['Arrest Date'])
df_cleansed['Year'] = df_cleansed['Date'].dt.year

In [6]:
df_cleansed.to_csv(r'Cleansed.csv')

In [8]:
df_cleansed.describe()

,Reporting District,Age,Year
count,1.324973e+06,1.324973e+06,1.324973e+06
mean,1.068565e+03,3.422401e+01,2.013976e+03
std,6.185899e+02,1.360864e+01,2.801748e+00
min,1.000000e+02,0.000000e+00,2.010000e+03
25%,5.240000e+02,2.300000e+01,2.012000e+03
50%,1.113000e+03,3.200000e+01,2.014000e+03
75%,1.546000e+03,4.500000e+01,2.016000e+03
max,2.199000e+03,9.700000e+01,2.020000e+03


In [12]:
#create year, month, day, day of week columns

def findYear(date):
    year = datetime.datetime.strptime(date, '%m/%d/%Y').year
    return(year)

df_cleansed['Year'] = df_cleansed['Arrest Date'].apply(findYear)

def findMonth(date):
    month = datetime.datetime.strptime(date, '%m/%d/%Y').month
    return(month)

df_cleansed['Month'] = df_cleansed['Arrest Date'].apply(findMonth)

def findDay(date):
    day = datetime.datetime.strptime(date, '%m/%d/%Y').day
    return(day)

df_cleansed['Day'] = df_cleansed['Arrest Date'].apply(findDay)

def findDay(date): 
    day = datetime.datetime.strptime(date, '%m/%d/%Y').weekday() 
    return (calendar.day_name[day]) 

#df_cleansed['Day of Week'] = df_cleansed['Arrest Date'].apply(findDay)

def findNDay(date): 
    day = datetime.datetime.strptime(date, '%m/%d/%Y').weekday() 
    return (day) 

df_cleansed['N Day of Week'] = df_cleansed['Arrest Date'].apply(findNDay)

#check results
df_cleansed

,Arrest Date,Area Name,Reporting District,Age,Sex Code,Descent Code,Charge Group Description,Arrest Type Code,Charge,Charge Description,NewTime,Street,CrossStreet,Date,Year,Month,Day,N Day of Week
0,05/03/2019,Southeast,1802,23,F,B,NaN,M,653.22 PC,NaN,17:00:00,91ST,FIGUEROA,2019-05-03,2019,5,3,4
1,04/29/2019,West LA,842,41,M,H,Robbery,F,211PC,ROBBERY,10:40:00,WILSHIRE BL,NaN,2019-04-29,2019,4,29,0
2,11/23/2019,Hollenbeck,457,23,F,H,Robbery,F,211PC,ROBBERY,18:30:00,FICKETT,CINCINNATI,2019-11-23,2019,11,23,5
3,04/30/2019,Hollywood,663,27,M,O,Burglary,F,459PC,BURGLARY,06:15:00,LA BREA,LEXINGTON,2019-04-30,2019,4,30,1
4,04/30/2019,Van Nuys,901,2,F,H,NaN,D,300(B)WIC,NaN,11:00:00,RAYMER,SEPULVEDA BL,2019-04-30,2019,4,30,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1324968,04/27/2019,Topanga,2125,22,M,H,Driving Under Influence,M,23152(G)VC,DUI OF ALCOHOL & DRUG,23:55:00,STRATHERN,CANOGA,2019-04-27,2019,4,27,5
1324969,04/27/2019,Central,161,30,M,W,Other Assaults,F,69PC,OBSTRUCT/RESIST EXECUTIVE OFFICER,18:29:00,HARBOR FY,4TH ST,2019-04-27,2019,4,27,5
1324970,04/27/2019,Devonshire,1797,42,M,H,Aggravated Assault,F,273.5(A)PC,CORPORAL INJURY ON SPOUSE/COHABITANT/ETC,23:15:00,RUBIO AV,NaN,2019-04-27,2019,4,27,5
1324971,04/27/2019,Southeast,1832,38,F,H,Aggravated Assault,F,273.5(A)PC,CORPORAL INJURY ON SPOUSE/COHABITANT/ETC,15:55:00,E 104TH ST,NaN,2019-04-27,2019,4,27,5
